In [5]:
import urllib
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [3]:
itas2010 = 'http://www.itas2010.iitp.ru/search.html'
itas2011 = 'http://www.itas2011.iitp.ru/search.html'
itas2012 = 'http://www.itas2012.iitp.ru/search.html'
itas2013 = 'http://itas2013.iitp.ru/ru/search.html'
itas2014 = 'http://itas2014.iitp.ru/ru/search.html'
itas2015 = 'http://itas2015.iitp.ru/ru/search.html'
itas2016 = 'http://itas2016.iitp.ru/ru/search.html'
itas2017 = 'http://itas2017.iitp.ru/papers/'

In [22]:
def load_authors_titles(url, year):
    import urllib
    from bs4 import BeautifulSoup
    
    response = urllib.request.urlopen(url)
    if response.status == 200:
        soup = BeautifulSoup(response, 'html.parser')
        
        titles = soup.find_all("a", {"class": "PaperTitle"})
        authors = soup.find_all("span", {"class": "PaperAuthors"})
        
        authors_list = []
        titles_list = []
        
        for auth, title in zip(authors, titles):
            authors_list.append(auth.get_text())
            titles_list.append(title.get_text())
            
        years = [year]*len(authors_list)
        
        return authors_list, titles_list, years
            
    else:
        print(response)
        return -1, -1, -1
    
    
def get_itas2017():    
    def get_author(text):
        try:
            all_authors, other = text.split('"other_authors"')
            other_authors, other = other.split('"first_author"')
            first_author, other = other.split('"platform"')
            other_authors, pdf= other_authors.split('"pdf"')
            other = bytes(other_authors, 'utf-8').decode('unicode_escape')
            first = bytes(first_author, 'utf-8').decode('unicode_escape')

            all_authors = []
            for f in first.split('"'):
                if len(f) > 5:
                    all_authors.append(f)

            for o in other.split('"'):
                if len(o) > 7:
                    all_authors.append(o)
            return all_authors
        except:
            return []
    data = []
    authors=[]
    response = urllib.request.urlopen(itas2017)
    soup = BeautifulSoup(response, 'html.parser')
    papers = soup.text.split('"id"')
    
    for idx,p in enumerate(papers):
        try:
            info = p.split('"abstract"')[1].split('"title"')[1]
            info2 = info.split('"all_authors"')
            title = bytes(info2[0], 'utf-8').decode('unicode_escape')
            for t in title.split('"'):
                if len(t)> 7:
                    main_title = t
            data.append(main_title)#(title)
            first_author = get_author(info2[1])
            authors.append(first_author)
        except:
            pass
        
    #authors[40], authors[42], authors[77], authors[102]
    #([], [], [], [])
    
    authors[40] = ['Анастасия Жарикова']
    authors[42] = ['Igor Zhilin', 'Victor Zyablov']
    authors[77] = ['Дарья Николаева', 'Софья Гарушянц']
    authors[102] = ['Nadezhda Terekhanova']
    
    _authors = []
    for auth in authors:
        _authors.append(', '.join(auth))
    
    return data, _authors


def process_authors(auth):
    a = []
    auth = auth.strip()
    for x in auth.split(','):
        a.append(x.strip())
    return a

## Store info in DataFrame, combine all years

In [6]:
authors, titles, years = load_authors_titles(itas2010, 2010)

itas_all = pd.DataFrame(data={'authors': authors,
                          'titles':titles,
                          'year':years}, 
                    columns=[['authors', 'titles', 'year']])

for url,year in zip([itas2011, itas2012, itas2013, itas2014, itas2015, itas2016],
                   [2011, 2012, 2013, 2014, 2015, 2016]):

    authors, titles, years = load_authors_titles(url, year)

    itas = pd.DataFrame(data={'authors': authors,
                              'titles':titles,
                              'year':years}, 
                        columns=[['authors', 'titles', 'year']])

    itas_all = pd.concat([itas_all, itas])

In [23]:
titles, authors = get_itas2017()

In [27]:
years = [2017]*113
itas = pd.DataFrame(data={'authors': authors,
                              'titles':titles,
                              'year':years}, 
                        columns=[['authors', 'titles', 'year']])
itas_all = pd.concat([itas_all, itas])

In [29]:
itas_all.drop_duplicates(inplace=True)

In [30]:
authors = []
titles = []
years = []
for row in itas_all.iterrows():
    #print(type(row))
    #print(row[1].authors)
    article_authors = process_authors(row[1].authors)
    for auth in article_authors:
        authors.append(auth)
        titles.append(row[1].titles)
        years.append(row[1].year)

In [31]:
itas = pd.DataFrame(data={'author':authors,
                   'title':titles,
                   'year':years}, columns=[['author', 'title', 'year']])

In [34]:
itas.iloc[[678, 1114, 1199, 1964]].values

array([['',
        'Количественное исследование активности центральных зрительных нейронов лягушки в зависимости от угловой скорости движения внешнего раздражителя ',
        2012],
       ['',
        'НКРЯ: о некоторых различиях в описании морфологии разных частей корпуса и об автоматическом построении синтаксических структур с учетом заранее известных морфологических характеристик ',
        2014],
       ['',
        'Использование критерия Отсу в пространстве геометрических разрезов на примере задачи поиска линии горизонта ',
        2014],
       ['',
        'Reasoning over Knowledge-based Generation of Situations in Context Spaces to Reduce Food Waste ',
        2016]], dtype=object)

In [56]:
# itas[itas.title == 'Количественное исследование активности центральных зрительных нейронов лягушки в зависимости от угловой скорости движения внешнего раздражителя ']
# Третий автор Владимир Антонинович Бастаков
itas = itas.set_value(678, 'author', 'Бастаков В.А.')

In [52]:
# itas[itas.title == 'НКРЯ: о некоторых различиях в описании морфологии разных частей корпуса и об автоматическом построении синтаксических структур с учетом заранее известных морфологических характеристик ']
# Третий автор Виктор Сизов
itas = itas.set_value(1114, 'author', 'Сизов В.')

In [54]:
# itas[itas.title == 'Использование критерия Отсу в пространстве геометрических разрезов на примере задачи поиска линии горизонта ']
# Третий автор Дарья Крохина
itas = itas.set_value(1199, 'author', 'Крохина Д.')

In [55]:
# itas[itas.title == 'Reasoning over Knowledge-based Generation of Situations in Context Spaces to Reduce Food Waste ']
# Четвертый автор Sylvain Kubler
itas = itas.set_value(1964, 'author', 'Sylvain Kubler')

In [57]:
itas.head()

,author,title,year
0,Алексей Адов,Оптимальный речевой кодек для VoIP в децентрал...,2010
1,Кирилл Андреев,Оптимальный речевой кодек для VoIP в децентрал...,2010
2,Павел Бойко,Оптимальный речевой кодек для VoIP в децентрал...,2010
3,Александр Панчин,Зависимость мутационных процессов в геноме чел...,2010
4,Сергей Митрофанов,Зависимость мутационных процессов в геноме чел...,2010


## Process authors -> split into names and surnames

In [70]:
def split_author_names(author):
    
    name = author.split()
    
    if len(name) == 1:
        first_name = author
        second_name = author
        
        return first_name, second_name
    elif len(name) == 3:
        first_name = author
        second_name = author
        
        return first_name, second_name
   
    # if len(name == 2)
    
    name1 = name[0]
    name2 = name[1]

    if '.' in name1:
        first_name = name1
        second_name = name2
    elif '.' in name2:
        second_name = name1
        first_name = name2
    else:
        first_name = name1
        second_name = name2
        
    return first_name, second_name

In [71]:
# for idx, val in enumerate(itas.author.values):
# #     if len(val.split(' ')) != 2:
# #         print(idx, val, len(val.split(' ')))
#     print(val.split(' '))

In [72]:
first_name = []
second_name = []
for auth in itas.author.values:
    name1, name2 = split_author_names(auth)
    first_name.append(name1)
    second_name.append(name2)
    
itas['Name'] = first_name
itas['Surname'] = second_name

In [75]:
itas[['Name', 'Surname']].drop_duplicates().sort_values(by=['Surname', 'Name'])

,Name,Surname
1553,Hadeel,Abdah
1557,Ekaterina,Abushova
1564,Evgeny,Akkuratov
1574,Evgeni,Aksenov
1571,Vladimir,Aksenov
2191,Amin,Al-Ka'bi
1587,Mohammed,Al-Mardi
1593,Abdulwahab,Al-odhari
1679,Smirnov,Alexander
1788,Alexander P. Lavrov,Alexander P. Lavrov


In [78]:
itas.to_excel('itas_2010-2017_v2.xls')

In [79]:
itas.head()

,author,title,year,Name,Surname
0,Алексей Адов,Оптимальный речевой кодек для VoIP в децентрал...,2010,Алексей,Адов
1,Кирилл Андреев,Оптимальный речевой кодек для VoIP в децентрал...,2010,Кирилл,Андреев
2,Павел Бойко,Оптимальный речевой кодек для VoIP в децентрал...,2010,Павел,Бойко
3,Александр Панчин,Зависимость мутационных процессов в геноме чел...,2010,Александр,Панчин
4,Сергей Митрофанов,Зависимость мутационных процессов в геноме чел...,2010,Сергей,Митрофанов


In [81]:
itas[itas.title == 'Сдвиговое и полусдвиговое преобразование Хаффа: генерация быстрых вычислительных схем ']

,author,title,year,Name,Surname
115,Семен Карпенко,Сдвиговое и полусдвиговое преобразование Хаффа...,2010,Семен,Карпенко
116,Валерий Соколов,Сдвиговое и полусдвиговое преобразование Хаффа...,2010,Валерий,Соколов
117,Дмитрий Николаев,Сдвиговое и полусдвиговое преобразование Хаффа...,2010,Дмитрий,Николаев
177,Семен Карпенко,Сдвиговое и полусдвиговое преобразование Хаффа...,2010,Семен,Карпенко
178,Валерий Соколов,Сдвиговое и полусдвиговое преобразование Хаффа...,2010,Валерий,Соколов
179,Дмитрий Николае,Сдвиговое и полусдвиговое преобразование Хаффа...,2010,Дмитрий,Николае


In [84]:
itas_2 = pd.read_excel('itas_2010-2017_v3.xls')
# после ручной обработки

In [87]:
itas_2['author_id_new'] = itas_2.Name + ' ' + itas_2.Surname

In [92]:
itas_2 = itas_2.drop_duplicates()

In [93]:
itas_2.to_excel('data_rus_author_v4.xls')

In [94]:
# В данной версии содержатся все данные, (почти) все авторы приведены к русскому имени,
# к одинаковой форме (либо имя + фамилия либо инициалы + фамилия) .

In [96]:
a = itas_2.author_id_new.values

In [98]:
auth, counts = np.unique(a, return_counts=True)

In [99]:
order = np.argsort(counts)[::-1]
auth = auth[order]
counts = counts[order]

In [100]:
for a, c in zip(auth, counts):
    print(a,c)

Евгений Хоров 51
Г.А. Базыкин 47
Андрей Миронов 43
Дмитрий Николаев 32
А.С. Кондрашов 28
В.В. Зяблов 26
Михаил Беляев 25
Евгений Бурнаев 24
Александр Фаворов 23
Андрей Ляхов 20
Михаил Гельфанд 18
Елена Ставровская 17
Артем Красилов 16
Дмитрий Родионов 15
Максим Панов 15
Антон Кирьянов 14
П.В. Приходько 13
Екатерина Храмеева 13
С.А. Науменко 13
Павел Рыбин 12
Юрий Панчин 12
И.В. Жилин 12
Алексей Фролов 11
Ирина Солопова 11
Виктор Селионов 11
Надежда Тереханова 11
Дмитрий Банков 11
Александр Сафонов 11
Федор Иванов 11
Юлия Додонова 11
А.В. Гасников 11
Ольга Бочкарева 11
Борис Миллер 11
Иван Коноваленко 10
Александр Миллер 10
Владимир Спокойный 10
И.В. Вьюгин 10
Юрий Янович 10
Алексей Крещук 10
Степан Денисов 9
А.В. Селиверстов 9
Руслан Солдатов 9
А.С. Иванов 9
Карен Степанян 9
В.А. Любецкий 9
Владимир Сеплярский 9
Дмитрий Жванский 9
Владимир Потапов 8
Мария Логачева 8
Антон Григорьев 8
С.В. Виноградова 8
П.В. Шелякин 8
Александр Панчин 8
Алексей Пенин 8
Александр Кротов 8
В.Л. Талис 7
Зо